In [1]:
import pytorch_lightning as pl

pl.__version__

'1.5.7'

In [2]:
import torch

x = torch.arange(0, 12).reshape((-1, 3))
print(x)
print(x.tile((1, 2)))
y = torch.tensor([[1, 2], [3, 4]])
torch.tile(y, (2, 1))

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
tensor([[ 0,  1,  2,  0,  1,  2],
        [ 3,  4,  5,  3,  4,  5],
        [ 6,  7,  8,  6,  7,  8],
        [ 9, 10, 11,  9, 10, 11]])


tensor([[1, 2],
        [3, 4],
        [1, 2],
        [3, 4]])

In [21]:
import h5py
import numpy as np

path = '/home/research/research/modeling/weather_forecast/gwd/data/raw/2015-01-01.h5'
with h5py.File(path, "r") as file:
    print(file.keys())
    x = file['/x'][:]
    print(x.shape)
    print(x[:,0,1])
    y = file['/y'][:]
    print(y.shape)
x = np.reshape(x, (191, -1)).T
y = np.reshape(y, (126, -1)).T
print(x.shape)

<KeysViewHDF5 ['x', 'y']>
(191, 36, 65440)
[-1.6986801e+01 -2.9156265e+00 -3.1320343e+00 -2.7499237e+00
  8.0800705e+00  3.4188385e+01  7.1976273e+01  8.8823280e+01
  1.0028650e+02  1.0535427e+02  1.0833929e+02  1.0589876e+02
  1.0085615e+02  9.0348206e+01  8.4133598e+01  8.2608009e+01
  8.0453987e+01  7.5591057e+01  6.2250603e+01  5.2632618e+01
  4.5269985e+01  3.8062721e+01  3.2207039e+01  2.4713226e+01
  1.8252552e+01  1.4278503e+01  1.1508541e+01  8.6058388e+00
  5.9360199e+00  5.3372307e+00  4.0759277e+00  2.8003273e+00
  1.9899139e+00  1.4549828e+00  1.1292419e+00  8.0082703e-01
  1.2159748e+00  3.0609093e+00  5.1550217e+00  5.5192490e+00
  3.7863655e+00  2.1063080e+00  2.0110779e+00  3.5111885e+00
  5.7553101e+00  7.1366615e+00  6.9363861e+00  6.3675060e+00
  6.7905712e+00  7.6677017e+00  8.7694016e+00  1.0370247e+01
  1.3159538e+01  1.5086502e+01  1.6129131e+01  1.6455692e+01
  1.6858810e+01  1.7366577e+01  1.7568523e+01  1.7762718e+01
  1.7735500e+01  1.7693157e+01  1.7211395e

In [20]:
print(x[1])

[-1.6986801e+01 -2.9156265e+00 -3.1320343e+00 -2.7499237e+00
  8.0800705e+00  3.4188385e+01  7.1976273e+01  8.8823280e+01
  1.0028650e+02  1.0535427e+02  1.0833929e+02  1.0589876e+02
  1.0085615e+02  9.0348206e+01  8.4133598e+01  8.2608009e+01
  8.0453987e+01  7.5591057e+01  6.2250603e+01  5.2632618e+01
  4.5269985e+01  3.8062721e+01  3.2207039e+01  2.4713226e+01
  1.8252552e+01  1.4278503e+01  1.1508541e+01  8.6058388e+00
  5.9360199e+00  5.3372307e+00  4.0759277e+00  2.8003273e+00
  1.9899139e+00  1.4549828e+00  1.1292419e+00  8.0082703e-01
  1.2159748e+00  3.0609093e+00  5.1550217e+00  5.5192490e+00
  3.7863655e+00  2.1063080e+00  2.0110779e+00  3.5111885e+00
  5.7553101e+00  7.1366615e+00  6.9363861e+00  6.3675060e+00
  6.7905712e+00  7.6677017e+00  8.7694016e+00  1.0370247e+01
  1.3159538e+01  1.5086502e+01  1.6129131e+01  1.6455692e+01
  1.6858810e+01  1.7366577e+01  1.7568523e+01  1.7762718e+01
  1.7735500e+01  1.7693157e+01  1.7211395e+01 -1.0831318e+01
 -1.1767979e+01 -8.04697

In [4]:
x.shape

(2355840, 191)

In [106]:
import torch.nn as nn
import torch

class Reshape(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        t0 = torch.reshape(x[:, :3 * 63], [-1, 3, 63])
        t1 = torch.tile(x[:, 3 * 63:].unsqueeze(2), (1, 1, 63))

        return torch.cat((
            t0, t1,
            # torch.transpose(t0, -2, -1),
            # torch.transpose(t1, -2, -1)
        ), dim=1)

In [150]:
# model = nn.Sequential(
#     Reshape(),
#     nn.Conv1d(5, 16, 3, padding=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool1d(2),
#     # nn.Conv1d(16, 32, 3),
#     # nn.ReLU(inplace=True),
#     # nn.MaxPool1d(2),
#     nn.Flatten(),
#     nn.Linear(16, 126),
#     nn.Linear(126, 126)
# )

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.ModuleList([
            Reshape(),
            nn.Conv1d(5, 16, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((32 - 3) // 2 * 32, 126),
            nn.Linear(126, 126)
        ])
        
    def forward(self, x):
        for module in self.model:
            x = module(x)
            # print(x.shape)
        return x
    
model = Model()

In [151]:
29 // 2

14

In [152]:
x0 = torch.tensor(x[0:1, :])

In [153]:
model(x0)

torch.Size([1, 5, 63])
torch.Size([1, 16, 63])
torch.Size([1, 16, 63])
torch.Size([1, 16, 31])
torch.Size([1, 32, 29])
torch.Size([1, 32, 29])
torch.Size([1, 32, 14])
torch.Size([1, 448])
torch.Size([1, 126])
torch.Size([1, 126])


tensor([[-6.1728e+01, -1.0866e+03, -4.9883e+03,  1.4622e+03,  5.6056e+03,
          3.1360e+03,  7.0969e+01, -5.1658e+03, -5.9838e+03,  3.6009e+03,
          3.8060e+03, -2.5862e+03,  4.5961e+03, -5.3451e+02,  8.7825e+02,
          2.8244e+03,  5.3087e+00, -2.7571e+03, -1.2671e+03,  4.9432e+03,
          1.7405e+03, -2.7600e+03,  1.6884e+02,  4.1402e+03,  4.0978e+03,
         -1.4758e+03, -6.3077e+03,  2.8228e+03, -2.6621e+03,  3.0024e+02,
          1.6726e+03, -1.3803e+01,  1.8552e+03,  2.9832e+03,  3.2422e+03,
          3.9024e+02,  6.8351e+01,  1.1052e+01,  8.3694e+02,  1.2468e+03,
         -4.8685e+03, -2.0873e+03,  4.5530e+03, -1.8417e+03,  7.0575e+02,
         -2.1839e+03, -1.5145e+03, -5.1195e+02,  1.5220e+03, -3.1714e+03,
         -2.0969e+02,  2.5381e+03,  8.1095e+02, -1.9591e+03,  1.6327e+03,
          3.4591e+03,  3.8360e+03, -3.5886e+03, -3.4115e+02,  1.2770e+03,
         -2.8518e+03, -4.7503e+03,  4.1841e+03, -1.4247e+03, -1.7041e+03,
          5.3519e+03, -8.3556e+02,  5.

In [ ]:
def build_model(input_shape,
                output_shape,
                features_root,
                filter_size,
                pool_size,
                n_levels,
                neurons,
                mean,
                var,
                l_rate):
    inputs = Input(shape=(input_shape,))
    
    norm = Normalization(mean=mean, variance=var**2)
    scaled = norm(inputs)

    cnn = ReshapeAndBroadcast()(scaled)
    for i in range(n_levels):
        n_features = (2**i) * features_root
        cnn = Conv1D(filters=n_features, kernel_size=filter_size, activation="relu")(cnn)
        cnn = MaxPool1D(pool_size=pool_size)(cnn)
    flat = Flatten()(cnn)
    dense = Dense(neurons, activation="relu")(flat)
    outputs = Dense(output_shape, activation="linear")(dense)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=l_rate), loss="mse", 
                  metrics=[RootMeanSquaredError(name="rmse"), r2_score])
    model.summary()
    
    return model

config = {
    'input_shape' : 191,
    'output_shape' : 126,
    'features_root' : 8,
    'filter_size' : 3,
    'pool_size' : 2,
    'n_levels' : 2,
    'neurons' : 126,
    'mean' : x_mean,
    'var' : x_std,
    'l_rate' : 1.e-4,
}

In [21]:
in_path = '/home/research/research/modeling/weather_forecast/gwd/data/raw/2015-01-01.h5'
with h5py.File(in_path, "r") as file:
    x = file['/x'][:]
    y = file['/y'][:]
    
x = np.reshape(x, (191, -1)).T
y = np.reshape(y, (126, -1)).T
out_path = '/home/research/research/modeling/weather_forecast/gwd/data/processed/2015-01-01.h5'
with h5py.File(out_path, 'w') as file:
    file['/x'] = x
    file['/y'] = y

In [27]:
import os

in_path = '/home/research/research/modeling/weather_forecast/gwd/data/processed/2015-01-01.h5'
os.path.split(in_path)

('/home/research/research/modeling/weather_forecast/gwd/data/processed',
 '2015-01-01.h5')

In [24]:
in_path = '/home/research/research/modeling/weather_forecast/gwd/data/processed/2015-01-01.h5'
with h5py.File(in_path, "r") as file:
    x = file['/x'][4003]
    y = file['/y'][4003]
    
print(x, y)

[ 1.22631989e+01  2.88343735e+01  4.96179657e+01  7.10000763e+01
  7.73300705e+01  6.71883850e+01  6.49762726e+01  7.68232803e+01
  8.92864990e+01  9.38542709e+01  9.23392868e+01  8.46487579e+01
  6.66061478e+01  5.20982056e+01  4.48835983e+01  4.01080093e+01
  3.27039871e+01  2.63410568e+01  2.22506027e+01  2.16326180e+01
  2.25199852e+01  2.00627213e+01  1.68320389e+01  1.58382263e+01
  1.62525520e+01  1.77785034e+01  1.86335411e+01  1.78558388e+01
  1.64360199e+01  1.45872307e+01  1.44509277e+01  1.69253273e+01
  1.82399139e+01  1.74549828e+01  1.63792419e+01  1.53008270e+01
  1.40909748e+01  1.33109093e+01  1.64050217e+01  2.27692490e+01
  2.55363655e+01  2.47313080e+01  2.40110779e+01  2.45111885e+01
  2.55053101e+01  2.75116615e+01  3.43113861e+01  3.84925079e+01
  4.10405731e+01  4.52927017e+01  4.90194016e+01  5.08702469e+01
  5.09095383e+01  4.95865021e+01  4.78791313e+01  4.52056923e+01
  4.18588104e+01  3.76165771e+01  3.51935234e+01  3.52627182e+01
  3.54855003e+01  3.30681

In [16]:
2355840 / 128 * 18

331290.0

In [12]:
51 // 10

5

In [ ]:
#!/usr/bin/python3
# coding: utf-8

import glob
import os
import dask.array as da
import numpy as np
import tensorflow as tf
import xarray as xr

from tensorflow.distribute import MirroredStrategy
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import MaxPool1D, Conv1D, Dense, Flatten, Layer
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

class DataLoader:
        
    def __init__(self, root_path, processed_path, raw_files=None, batch_size=32, mode="train", shuffle=True, y_std_max=None):
        self.root_path = root_path
        self.processed_path = processed_path
        self.raw_files = raw_files
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        self.y_std_max = y_std_max
        self.raw_path = os.path.join(self.root_path, self.mode)
        self.x_storage_dir = os.path.join(self.processed_path, "batch_size_{}".format(self.batch_size), self.mode, "x_{}.npy".format(self.mode))
        self.y_storage_dir = os.path.join(self.processed_path, "batch_size_{}".format(self.batch_size), self.mode, "y_{}.npy".format(self.mode))
        
        if "process" in self.__class__.__dict__.keys():
            if not os.path.exists(self.x_storage_dir):
                os.makedirs(self.x_storage_dir)

            if not os.path.exists(self.y_storage_dir):
                os.makedirs(self.y_storage_dir)

            self._process()
        
        self.list_processed_files()
        
    def process(self):
        """Load, stack and reshape all the files"""
        
        print("Loading the dataset...")
        list_files = [os.path.join(self.raw_path, f) for f in self.raw_files]
        print(list_files)
        ds = xr.open_mfdataset(list_files, chunks=-1, combine="nested", concat_dim="concat_dim", parallel=True)
        x_ = da.moveaxis(ds.x.data, 1, -1).reshape(ds.x.shape[0]*ds.x.shape[2]*ds.x.shape[3], -1)
        y_ = da.moveaxis(ds.y.data, 1, -1).reshape(ds.y.shape[0]*ds.y.shape[2]*ds.y.shape[3], -1)
        
        self.sample_size = x_.shape[0]
        self.input_dim = x_.shape[1]
        self.output_dim = y_.shape[1]

        x_ = x_.rechunk((self.batch_size, self.input_dim))
        y_ = y_.rechunk((self.batch_size, self.output_dim))
        
        self.x_mean = x_.mean(axis=0).compute()
        self.x_std = x_.std(axis=0).compute()
        self.y_std_max = y_.std(axis=0).max().compute()
        with open(os.path.join(self.root_path, "normalization_parameters.npy"), 'wb') as f:
            np.save(f, self.x_mean)
            np.save(f, self.x_std)
            np.save(f, self.y_std_max)

        # Features
        x_ = x_.rechunk((self.batch_size, self.input_dim))
        print("Storing X array in npy files...")
        da.to_npy_stack(self.x_storage_dir, x_, axis=0)

        # Targets
        y_ = y_.rechunk((self.batch_size, self.output_dim))
        print("Storing Y array in npy files...")
        da.to_npy_stack(self.y_storage_dir, y_, axis=0)
        
    def _process(self):
        if (os.path.exists(self.x_storage_dir) and len(os.listdir(self.x_storage_dir)) != 0 and 
                os.path.exists(self.y_storage_dir) and len(os.listdir(self.y_storage_dir)) != 0):
            print("Raw files already processed...")
            return
        
        print("Processing...")
        
        self.process()
        
        print("Done!")
    
    def list_processed_files(self):
        self.processed_files = glob.glob(os.path.join(self.x_storage_dir, "*.npy"))
    
    def create_tf_dataset(self):
        print("Creating TF dataset...")
        dataset = tf.data.Dataset.from_tensor_slices(self.processed_files)
        dataset = (dataset
                   .map(lambda item: tf.py_function(self.read_npy_files, [item], Tout=(tf.float32, tf.float32)), num_parallel_calls=tf.data.AUTOTUNE)
                   .filter(lambda features, targets : tf.shape(features)[0] == self.batch_size)
                   .prefetch(tf.data.AUTOTUNE))
        print("Done !")
        return dataset
        
    def read_npy_files(self, xfilename):
        xfilename = xfilename.numpy().decode()
        yfilename = xfilename.replace('x_{}'.format(self.mode), 'y_{}'.format(self.mode))
        x = np.load(xfilename)
        y = np.load(yfilename)
        
        if self.y_std_max is not None:
            y = y / self.y_std_max
        
        return (x.astype(np.float32), y.astype(np.float32))

class ReshapeAndBroadcast(Layer):
    """
    Define a custom layer that takes an array (Nx191), reshape it to (Nx63x5)
    by broadcasting the two surface quantities (pressure and geopotential).

    Methods
    -------
    call():
        Performs the logic of applying the layer to the input tensors:
            - reshape (batch_sizex63x3)
            - broadcast (batch_sizex63x2)
            - concatenate
    """

    def call(self, inputs, **kwargs):
        t0 = tf.reshape(inputs[:, :3 * 63], [-1, 3, 63])
        t1 = tf.tile(tf.expand_dims(inputs[:, 3 * 63:], axis=-1), [1, 1, 63])

        return tf.concat((
            tf.transpose(t0, perm=[0, 2, 1]),
            tf.transpose(t1, perm=[0, 2, 1])), 2)

def r2_score(y_true, y_pred):
    ss_res = tf.keras.backend.sum((y_true - y_pred) ** 2)
    ss_tot = tf.keras.backend.sum((y_true - tf.keras.backend.mean(y_true)) ** 2)
    return 1 - ss_res / (ss_tot + tf.keras.backend.epsilon())

def build_model(input_shape,
                output_shape,
                features_root,
                filter_size,
                pool_size,
                n_levels,
                neurons,
                mean,
                var,
                l_rate):
    inputs = Input(shape=(input_shape,))
    
    norm = Normalization(mean=mean, variance=var**2)
    scaled = norm(inputs)

    cnn = ReshapeAndBroadcast()(scaled)
    for i in range(n_levels):
        n_features = (2**i) * features_root
        cnn = Conv1D(filters=n_features, kernel_size=filter_size, activation="relu")(cnn)
        cnn = MaxPool1D(pool_size=pool_size)(cnn)
    flat = Flatten()(cnn)
    dense = Dense(neurons, activation="relu")(flat)
    outputs = Dense(output_shape, activation="linear")(dense)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=l_rate), loss="mse", 
                  metrics=[RootMeanSquaredError(name="rmse"), r2_score])
    model.summary()
    
    return model

def main():
    home_path = os.path.join(os.path.expanduser('~'))
    data_path = os.path.join(home_path, "data", "nogwd", "raw")
    save_path = os.path.join(home_path, "data", "nogwd", "processed", "chunked_files")
    model_path = os.path.join(home_path, "data", "models")
    
    model_name = "mlp"
    
    BATCH_SIZE = 256
    EPOCHS = 200
    checkpoint_path = os.path.join(home_path, "checkpoints", "{}.ckpt".format(model_name))
    
    list_files_train = [
        "2015-01-01.h5",
        "2015-01-31.h5",
        "2015-03-02.h5",
        "2015-04-01.h5",
        "2015-05-01.h5",
        "2015-05-31.h5",
        "2015-06-30.h5",
        "2015-07-30.h5",
        "2015-08-29.h5",
        "2015-09-28.h5",
        "2015-10-28.h5",
        "2015-11-27.h5",]
    
    params = {
        "batch_size" : BATCH_SIZE,
        "shuffle" : True,
    }
    
    # Create the training dataset
    train = DataLoader(data_path, save_path, list_files_train, **params, mode="train").create_tf_dataset()
    
    # Create the validation dataset
    list_files_val = [
        "2016-02-25.h5",
        "2016-06-24.h5",
        "2016-12-21.h5"
        ]
    val = DataLoader(data_path, save_path, list_files_val, **params, mode="val").create_tf_dataset()
    
    with open(os.path.join(data_path, "normalization_parameters.npy"), "rb") as f:
        x_mean = np.load(f)
        x_std = np.load(f)

    config = {
        'input_shape' : 191,
        'output_shape' : 126,
        'features_root' : 8,
        'filter_size' : 3,
        'pool_size' : 2,
        'n_levels' : 2,
        'neurons' : 126,
        'mean' : x_mean,
        'var' : x_std,
        'l_rate' : 1.e-4,
    }
    
    strategy = MirroredStrategy()
    with strategy.scope():
        my_model = build_model(**config)
    
    # Keras callbacks
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,
                                 monitor='val_rmse', mode='min', save_best_only=True)
    
    earlystopping = EarlyStopping(monitor='val_rmse', patience=5)
    
    tensorboard = TensorBoard(log_dir=os.path.join(home_path, "logs", model_name),
                              write_images=False,
                              write_graph=True,
                              histogram_freq=1,
                              update_freq="batch",
                              profile_batch=2,
    )
    
    history = my_model.fit(train,
                           validation_data=val,
                           batch_size=None,
                           epochs=EPOCHS,
                           verbose=1,
                           callbacks=[
                               earlystopping,
                               # checkpoint,
                               # tensorboard
                               ],
    )
    
    # Saving history (to plot loss curves)
    np.save(file=os.path.join(home_path, "history", "wf-nogwd-history-{}.npy".format(model_name)), 
            arr=history.history)
    
    # Saving the model at last epoch (not especially the best model)
    my_model.save(os.path.join(model_path, "wf-nogwd-model-{}.h5".format(model_name)))
    
    print("Training done !")

if __name__ == "__main__":
    main()

In [2]:
import h5py
import torch

path = '/home/research/research/modeling/weather_forecast/gwd/data/stats.h5'

with h5py.File(path, 'r') as file:
    x_mean = torch.tensor(file['/x_mean'])
    y_mean = torch.tensor(file['/y_mean'])
    x_std = torch.tensor(file['/x_std'])
    y_std = torch.tensor(file['/y_std'])

In [3]:
x_mean

tensor([ 1.6765e+00, -1.1890e+00, -2.8733e+00, -4.2150e+00, -4.7127e+00,
        -5.3416e+00, -7.2468e+00, -1.0985e+01, -1.4382e+01, -1.5157e+01,
        -1.3655e+01, -1.1488e+01, -9.1906e+00, -6.9320e+00, -4.7353e+00,
        -2.7538e+00, -1.5186e+00, -1.3258e+00, -2.0570e+00, -3.4567e+00,
        -5.3086e+00, -7.1967e+00, -8.5176e+00, -8.8866e+00, -8.3621e+00,
        -7.2705e+00, -5.9027e+00, -4.4026e+00, -2.8475e+00, -1.3116e+00,
         1.3604e-01,  1.4448e+00,  2.5981e+00,  3.6532e+00,  4.7253e+00,
         5.9303e+00,  7.2647e+00,  8.6301e+00,  9.9276e+00,  1.1111e+01,
         1.2175e+01,  1.3131e+01,  1.3970e+01,  1.4674e+01,  1.5221e+01,
         1.5591e+01,  1.5782e+01,  1.5807e+01,  1.5691e+01,  1.5452e+01,
         1.5104e+01,  1.4665e+01,  1.4143e+01,  1.3548e+01,  1.2891e+01,
         1.2185e+01,  1.1438e+01,  1.0658e+01,  9.8597e+00,  9.0714e+00,
         8.3104e+00,  7.5855e+00,  6.9153e+00,  2.8363e+00,  3.4845e+00,
         2.6375e+00,  2.0744e+00,  1.4482e+00,  1.1

In [3]:
preds = torch.arange(0, 12).reshape((-1, 3))
targets = torch.arange(1, 13).reshape((-1, 3))

In [4]:
preds, targets

(tensor([[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]]),
 tensor([[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9],
         [10, 11, 12]]))

In [43]:
from torchmetrics import R2Score
from sklearn.metrics import r2_score

In [44]:
r2score = R2Score()

In [45]:
target = torch.tensor([[0.5, 1], [-1, 1], [7, -6]])
preds = torch.tensor([[0, 2], [-1, 2], [8, -5]])
r2score = R2Score(num_outputs=2, multioutput='raw_values')
r2score(preds, target)

tensor([0.9654, 0.9082])

In [48]:
r2score = R2Score(num_outputs=2, multioutput='uniform_average')
r2score(preds, target)

tensor(0.9368)

In [39]:
target.shape

torch.Size([3, 2])

In [36]:
r2score = R2Score(num_outputs=3, multioutput='uniform_average')
r2score(preds, target)

tensor(0.6831)

In [14]:
target.shape

torch.Size([3, 2])

In [51]:
torch.sum(torch.ones((100,2)), dim=0)

tensor([100., 100.])

In [52]:
torch.sum(torch.ones((100,2)), dim=1)

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])